In [ ]:
import numpy as np
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
!pip install --upgrade imgaug
from imgaug import augmenters as iaa
import os

     |████████████████████████████████| 948 kB 7.7 MB/s 
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.4.0 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
model = tf.keras.models.load_model('drive/MyDrive/credit/credit_card_model (1).h5')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 83, 56, 10)        280       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 79, 52, 10)        2510      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 73, 46, 10)        4910      
_________________________________________________________________
flatten (Flatten)            (None, 33580)             0         
_________________________________________________________________
dense (Dense)                (None, 10)                335810    
Total params: 343,510
Trainable params: 343,510
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        print(filename)
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

In [ ]:
imgs = []
max_img_count = 1000
for i in range(10):
    digit = load_images_from_folder('drive/MyDrive/emnist/digit/'+str(i))
    imgs.append(digit)

bankOfChinaCUPPlatinum-3.png
beaFlyerWorld-6.png
ccbMasterCard-6.png
ccbMasterCard-7.png
bociCard-3.png
bocommGoGoal-5.png
ccbEye-7.png
citiOctopusGoldVisa-7.png
citiOctopusPlat-7.png
dahSingAltruist-4.png
dahSingAltruist-5.png
citiPremierMiles-7.png
citiPremierMiles-1.png
dahSingChickeeduck-4.png
dahSingChickeeduck-5.png
dahSingMileagePlus-3.png
dahSingMileagePlus-4.png
dahSingStudent-2.png
dbsPlatinum-1.png
fubonYATA-12.png
fubonYATA-1.png
dbsPlatinum-3.png
fuBonPlatVisa-1.png
dbsPlatinum-13.png
fuBonPlatVisa-12.png
fuBonTitanium-12.png
hangSengCUP-6.png
hangSengCUP-3.png
hangSengCUP-7.png
fubonYATAKUMAMOTO-12.png
hangSengHKDPlatCommCard-6.png
hangSengPlatinumVisa-7.png
hangSengPlatinumVisa-1.png
hangSengPlatinumVisa-2.png
hangSengPlatinumVisa-6.png
hsbcAdvance-9.png
hsbcAdvance-11.png
hsbcAdvance-7.png
hsbcAdvance-8.png
hsbcAdvance-10.png
hsbcAdvance-6.png
hangSengMpower-4.png
hangSengMpower-2.png
hangSengMUJIVisa-7.png
hangSengMUJIVisa-2.png
hangSengMUJIVisa-1.png
hangSengMUJIVisa-

In [ ]:
import random
max_img_count = 1000
imgs_equal = [] #holds equal number of images for all categories(0-9)
seq = iaa.Sequential([
    iaa.Affine(scale=(0.8, 1.0), translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}),
    # iaa.Affine(shear=(-10, 10))
    iaa.Add((-10, 10), per_channel=0.5),
    # iaa.Multiply((0.5, 1.5), per_channel=0.5),
    # iaa.BlendAlphaVerticalLinearGradient(
    #   iaa.TotalDropout(1.0),
    #   min_value=0.2, max_value=0.8),
    # iaa.MeanShiftBlur()

])
for number in imgs:
    count=0
    while(count < max_img_count):#generating filler data
        rand_img = random.sample(number, 1)[0]
        rows, cols, _ = rand_img.shape
        rotated = rand_img.copy()
        # matrix = cv.getRotationMatrix2D((cols,rows),random.randint(-10,10),1)
        # rotated = cv.warpAffine(rand_img,matrix,(cols,rows), flags=cv.INTER_LINEAR, borderMode=cv.BORDER_REPLICATE)
#         roatated = vertical_shift(rotate, )
        rotated = seq(images=rotated.reshape(-1, *rotated.shape))[0]
        # rotated = iaa.Affine(translate_px=(-10, 10), shear=(-10, 10))(images=rotated.reshape(-1, 85, 58, 3)).reshape(85, 58, 3)
        # rotated = iaa.Add()(images=rotate.reshape((-1, 85, 58, 3)))
        imgs_equal.append(rotated)
        count+=1

In [ ]:
len(imgs_equal)

10000

In [ ]:
imgs_resized = [] #holds resized images
for number in imgs_equal:
  imgs_resized.append(cv2.resize(number, dsize=(58, 85), interpolation=cv2.INTER_CUBIC))

In [ ]:
def onehotencode(index, n):
    return [1.0 if i == index else 0.0 for i in range(n)]

In [ ]:
labels = []
for i in range(10):
    for j in range(max_img_count):
        labels.append(onehotencode(i, 10))

In [ ]:
data = np.asarray(imgs_resized)/255 #VERY IMPORTANT TO NORMALIZE PIXELS FROM 0-255 to 0-1
labels = np.asarray(labels)
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.3)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(7000, 85, 58, 3) (3000, 85, 58, 3) (7000, 10) (3000, 10)


In [ ]:
from keras import backend as K
K.set_value(model.optimizer.learning_rate, 0.0001)
model.fit(x_train, y_train, 10, 10, validation_data =(x_test, y_test), verbose = 1)

Epoch 1/10
 57/700 [=>............................] - ETA: 1:57 - loss: 1.2877 - accuracy: 0.8263

KeyboardInterrupt: ignored

In [ ]:
TEST_FOLDER_NAME = 'drive/MyDrive/credit/test (copy)/'
total = 0
correct = 0
confusion_matrix = np.zeros((10, 10))
for filename in os.listdir(TEST_FOLDER_NAME):
  image = cv2.imread(TEST_FOLDER_NAME + filename)
  # image = format_image(image)
  image = cv2.resize(image, (58, 85))
  # image = expand_image(image)
  predict = model.predict(image.reshape((1, 85, 58, 3)))
  expected = int(filename.split('.')[0].split('_')[-1])
  print("{:<35} {} {} = {} ({})".format(filename, np.argmax(predict), expected, predict[0, np.argmax(predict)],predict[0][expected]))
  total = total + 1
  correct = correct + (np.argmax(predict) == expected)
  confusion_matrix[np.argmax(predict), expected] = confusion_matrix[np.argmax(predict), expected] + 1
print("Result: {}/{}".format(correct, total))

3_bankOfCommYanChai_2.jpg           2 2 = 1.0 (1.0)
9_icbcAXAPlatinum_9.jpg             9 9 = 1.0 (1.0)
3_bocommGoGoal_2.jpg                2 2 = 1.0 (1.0)
7_hangSengPlatinumVisa_5.jpg        5 5 = 1.0 (1.0)
2_dahSingHelloKitty_2.jpg           2 2 = 1.0 (1.0)
2_bocommGoGoal_4.jpg                9 4 = 1.0 (0.0)
3_dahSingHelloKitty_1.jpg           1 1 = 1.0 (1.0)
3_citiPrestige_5.jpg                5 5 = 1.0 (1.0)
5_hangSengPlatinumVisa_4.jpg        4 4 = 1.0 (1.0)
4_ccbMasterCard_0.jpg               1 0 = 1.0 (0.0)
9_ccbTiramisu_7.jpg                 7 7 = 1.0 (1.0)
8_hangSengPlatinumVisa_3.jpg        3 3 = 1.0 (1.0)
8_aeonFerrari_8.jpg                 8 8 = 1.0 (1.0)
9_beaCorp_6.jpg                     6 6 = 1.0 (1.0)
3_beaCorp_5.jpg                     5 5 = 1.0 (1.0)
7_ccbTiramisu_5.jpg                 5 5 = 1.0 (1.0)
3_bocommPlatinum_2.jpg              2 2 = 1.0 (1.0)
13_icbcAXAPlatinum_8.jpg            8 8 = 1.0 (1.0)
10_icbcAXAPlatinum_9.jpg            9 9 = 1.0 (1.0)
14_icbcAXAPl

In [ ]:
!pip3 install tf-explain

     |████████████████████████████████| 43 kB 1.4 MB/s 


In [ ]:
from tf_explain.core.grad_cam import GradCAM

CREDIT_DATASET_FOLDER = "drive/MyDrive/credit/Credit Card Number Dataset/"

# tabby_cat_class_index = 0
explainer = GradCAM()
# Compute SmoothGrad on VGG16
for label in os.listdir(CREDIT_DATASET_FOLDER):
  # if os.path.exists(os.path.join(CREDIT_DATASET_FOLDER, label)):
  #   os.removed(os.path.join(CREDIT_DATASET_FOLDER, label))
  for filename in os.listdir(os.path.join(CREDIT_DATASET_FOLDER, label)):
    print(filename)
    image = cv2.imread(os.path.join(CREDIT_DATASET_FOLDER, label, filename))
    preprocess = cv2.resize(image, (58, 85)).astype(np.float32) / 255
    index = int(label)
    data = ([preprocess], None)
    grid = explainer.explain(data, model, index)
    explainer.save(grid, os.path.join('train_grad_cam', label), filename)

2_7.png
2_42.png
2_35.png
2_66.png
2_64.png
2_69.png
2_73.png
2_76.png
2_31.png
2_32.png
2_27.png
2_1.png
2_19.png
2_23.png
2_26.png
2_56.png
2_47.png
2_11.png
2_41.png
2_77.png
2_40.png
2_20.png
2_38.png
2_25.png
2_43.png
2_65.png
2_13.png
2_50.png
2_17.png
2_68.png
2_78.png
2_18.png
2_46.png
2_61.png
2_51.png
2_37.png
2_48.png
2_21.png
2_70.png
2_8.png
2_54.png
2_72.png
2_63.png
2_44.png
2_62.png
2_60.png
2_22.png
2_29.png
2_14.png
2_55.png
2_24.png
2_33.png
2_28.png
2_2.png
2_3.png
2_30.png
2_58.png
2_16.png
2_9.png
2_12.png
2_49.png
2_36.png
2_6.png
2_71.png
2_45.png
2_10.png
2_34.png
2_74.png
2_5.png
2_75.png
2_39.png
2_67.png
2_59.png
2_57.png
2_53.png
2_52.png
2_4.png
3_7.png
3_61.png
3_73.png
3_62.png
3_47.png
3_75.png
3_63.png
3_72.png
3_78.png
3_48.png
3_21.png
3_67.png
3_29.png
3_70.png
3_51.png
3_18.png
3_49.png
3_19.png
3_13.png
3_76.png
3_2.png
3_68.png
3_74.png
3_39.png
3_77.png
3_71.png
3_36.png
3_8.png
3_53.png
3_40.png
3_11.png
3_32.png
3_35.png
3_17.png
3_66.png
3_9.

In [ ]:
!zip -r train_grad_cam.zip train_grad_cam/

  adding: train_grad_cam/ (stored 0%)
  adding: train_grad_cam/0/ (stored 0%)
  adding: train_grad_cam/0/0_77.png (stored 0%)
  adding: train_grad_cam/0/0_26.png (deflated 0%)
  adding: train_grad_cam/0/0_37.png (stored 0%)
  adding: train_grad_cam/0/0_99.png (stored 0%)
  adding: train_grad_cam/0/0_92.png (deflated 0%)
  adding: train_grad_cam/0/0_39.png (stored 0%)
  adding: train_grad_cam/0/0_86.png (stored 0%)
  adding: train_grad_cam/0/0_18.png (stored 0%)
  adding: train_grad_cam/0/0_32.png (stored 0%)
  adding: train_grad_cam/0/0_38.png (stored 0%)
  adding: train_grad_cam/0/0_85.png (stored 0%)
  adding: train_grad_cam/0/0_104.png (stored 0%)
  adding: train_grad_cam/0/0_64.png (stored 0%)
  adding: train_grad_cam/0/0_70.png (stored 0%)
  adding: train_grad_cam/0/0_41.png (stored 0%)
  adding: train_grad_cam/0/0_69.png (stored 0%)
  adding: train_grad_cam/0/0_29.png (stored 0%)
  adding: train_grad_cam/0/0_6.png (stored 0%)
  adding: train_grad_cam/0/0_22.png (deflated 1%)
  ad